# Universal Sentence Encoder Baseline for XNLI task

In this notebook, we will walk you through the process of reproducing the Universal Sentence Encoder baseline for XNLI task

## Loading Required Modules

We start by loading the needed python libraries.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text
from tensorflow import keras
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder

## Loading Data

Using pandas, we can load and inspect the training, validation, and testing datasets as follows:

In [2]:
df_train = pd.read_csv("../../data/xnli/arabic_train.tsv", sep="\t")
df_test = pd.read_csv("../../data/xnli/arabic_dev.tsv", sep="\t")

Below we list the 5 first entries in the training data.

In [3]:
df_train.head()

,pairID,sentence1,sentence2,gold_label
0,4,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,لم أتحدث معه مرة أخرى.,contradiction
1,5,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,كنت مستاء جدا لدرجة أنني بدأت بالحديث معه مرة ...,entailment
2,6,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,دار بيننا حديث رائع.,neutral
3,7,واعتقدت أن ذلك شرف لي ، ولا يزال ، ولايزال ، ك...,لم أكن على علم بأنني لم أكن الشخص الوحيد الذي ...,neutral
4,8,واعتقدت أن ذلك شرف لي ، ولا يزال ، ولايزال ، ك...,كان لدي إنطباع أنني كنت الشخص الوحيد الذي لديه...,entailment


And last but not least, the first 5 entries in the test data.

In [4]:
df_test.head()

,pairID,sentence1,sentence2,gold_label
0,1,وقال، ماما، لقد عدت للمنزل.,اتصل بأمه حالما أوصلته حافلة المدرسية.,neutral
1,2,وقال، ماما، لقد عدت للمنزل.,لم ينطق ببنت شفة.,contradiction
2,3,وقال، ماما، لقد عدت للمنزل.,أخبر أمه أنه قد عاد للمنزل.,entailment
3,16,لم أعرف من أجل ماذا أنا ذاهب أو أي شىْ ، لذلك ...,لم أذهب إلى واشنطن من قبل، لذا عندما تم تكليفي...,neutral
4,17,لم أعرف من أجل ماذا أنا ذاهب أو أي شىْ ، لذلك ...,لقد عرفت بالضبط ما الذي احتجت أن أفعله عندما م...,contradiction


## Model Preparation

We start by setting the randomisation seed:

In [5]:
tf.random.set_seed(123)

Next we load the Universal Sentence Encoder (WARNING: This will download and cache a huge model of around 1 GB in size)

In [6]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

Then we define the input and output to the model:

In [7]:
sentence1_input = keras.Input(shape=512, name='sentence1')
sentence2_input = keras.Input(shape=512, name='sentence2')
label = keras.Input(shape=(1,), name='label')

This is followed by defining the structure of the network:

In [8]:
feat_1 = tf.abs(sentence1_input - sentence2_input)
feat_2 = sentence1_input*sentence2_input
features = tf.concat([sentence1_input, sentence2_input, feat_1, feat_2], 1)
logits = keras.layers.Dense(512*4, activation=tf.nn.tanh)(features)
logits = keras.layers.Dense(512*4, activation=tf.nn.tanh)(tf.concat([logits, features],1))
logits = keras.layers.Dense(512*4, activation=tf.nn.tanh)(tf.concat([logits, features],1))
logits = keras.layers.Dense(3, activation=tf.nn.softmax)(tf.concat([logits, features],1))

Then we construct and compile the model:

In [9]:
model = keras.Model(inputs=[sentence1_input, sentence2_input], outputs=logits)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Model Training

First we perpare the inputs and outputs to be fed to the model during training:

In [10]:
X1_train = embed(df_train["sentence1"])
X2_train = embed(df_train["sentence2"])
le = LabelEncoder()
le.fit(df_train["gold_label"])
Y_train = le.transform(df_train["gold_label"])

Next we fit the data:

In [11]:
model.fit([X1_train, X2_train],
          Y_train,
          epochs=5,
          batch_size=32)

Epoch 1/5
157/157 [==============================] - 14s 85ms/step - loss: 1.2351 - accuracy: 0.4275
Epoch 2/5
157/157 [==============================] - 13s 81ms/step - loss: 0.9382 - accuracy: 0.5648
Epoch 3/5
157/157 [==============================] - 11s 72ms/step - loss: 0.9140 - accuracy: 0.5912
Epoch 4/5
157/157 [==============================] - 12s 75ms/step - loss: 0.8820 - accuracy: 0.6007
Epoch 5/5
157/157 [==============================] - 12s 75ms/step - loss: 0.8462 - accuracy: 0.6353


## Diagnostic Dataset

Below we prepare the features for the test set as follows:

In [12]:
X1_test = embed(df_test["sentence1"])
X2_test = embed(df_test["sentence2"])
Y_test = le.transform(df_test["gold_label"])

Next we prepare the predictions

In [13]:
predictions_test = np.argmax(model.predict([X1_test, X2_test]),1)
df_test_sub = pd.DataFrame(data=le.inverse_transform(predictions_test), columns=["prediction"], index=df_test["pairID"])
df_test_sub.reset_index(inplace=True)

Then we save the prediction file

In [14]:
if not os.path.exists("predictions"):
    os.mkdir("predictions")
df_test_sub.to_csv("./predictions/xnli.tsv", index=False, sep="\t")

Next we perpare the features for each testset instance as follows:

In [15]:
diagnostic_data = pd.read_csv("../../private_datasets/diagnostic.tsv", sep="\t")

In [16]:
X1_dia = embed(diagnostic_data["sentence1"].tolist())
X2_dia = embed(diagnostic_data["sentence2"].tolist())
Y_dia = le.transform(diagnostic_data["gold_label"])

Then we predict the labels for each and evaluate the f1 score:

In [17]:
predictions = np.argmax(model.predict([X1_dia, X2_dia]),1)

We perpare the predictions as a pandas dataframe.

In [18]:
df_dia = pd.DataFrame(data=le.inverse_transform(predictions), columns=["prediction"], index=diagnostic_data["pairID"])
df_dia.reset_index(inplace=True)

In the final step, we save the predictions as required by the competition guidelines.

In [19]:
df_dia.to_csv("./predictions/diagnostic.tsv", index=False, sep="\t")